In [ ]:
!nvidia-smi

Fri Apr 12 08:55:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import pandas as pd

data = pd.read_csv("/content/sectionsdataset_withoutillustrations - Sheet1.csv")



In [ ]:
sections = data["Description"].values
sectionnames = data["Section"].values
sectionnumber = data["Number"].values

In [ ]:
len(sections)


506

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Vectorization of Sections#

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import BertTokenizer, BertModel
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',num_hidden_layers=24 , num_attention_heads=24 )

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Function to tokenize, remove stop words, and encode sentences
def encode_sentences(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        # Remove stop words
        cleaned_sentence = ' '.join([word for word in sentence.split() if word.lower() not in stop_words])
        cleaned_sentences.append(cleaned_sentence)

    # Tokenize and encode the cleaned sentences
    encoded_sentences = tokenizer(cleaned_sentences, return_tensors='pt', padding=True, truncation=True)

    return encoded_sentences

# Encode the sections
encoded_sections = encode_sentences(sections)

# Get BERT embeddings for the tokenized sections
def calculate_embeddings(encoded_sentences):
    with torch.no_grad():
        outputs = model(**encoded_sentences)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling of token embeddings
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=-1)  # L2 normalize
    return embeddings

#batch processing
def batch_process_sentences(sentences, batch_size=50):
    embeddings_list = []
    num_batches = (len(sentences) + batch_size - 1) // batch_size

    for i in range(num_batches):

        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(sentences))
        batch_sentences = sentences[start_idx:end_idx]
        print("Encoding sections from ",start_idx," to ",end_idx)
        # Encode batch of sentences
        encoded_sentences = encode_sentences(batch_sentences)

        # Calculate embeddings for batch
        embeddings = calculate_embeddings(encoded_sentences)
        embeddings_list.extend(embeddings)

    # Concatenate embeddings from all batches

    return embeddings_list


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.12.attention.output.LayerNorm.bias', 'bert.encoder.layer.12.attention.output.LayerNorm.weight', 'bert.encoder.layer.12.attention.output.dense.bias', 'bert.encoder.layer.12.attention.output.dense.weight', 'bert.encoder.layer.12.attention.self.key.bias', 'bert.encoder.layer.12.attention.self.key.weight', 'bert.encoder.layer.12.attention.self.query.bias', 'bert.encoder.layer.12.attention.self.query.weight', 'bert.encoder.layer.12.attention.self.value.bias', 'bert.encoder.layer.12.attention.self.value.weight', 'bert.encoder.layer.12.intermediate.dense.bias', 'bert.encoder.layer.12.intermediate.dense.weight', 'bert.encoder.layer.12.output.LayerNorm.bias', 'bert.encoder.layer.12.output.LayerNorm.weight', 'bert.encoder.layer.12.output.dense.bias', 'bert.encoder.layer.12.output.dense.weight', 'bert.encoder.layer.13.attention.output.LayerNorm.bias', 'bert

In [ ]:
embeddings = batch_process_sentences(sections)

Encoding sections from  0  to  50
Encoding sections from  50  to  100
Encoding sections from  100  to  150
Encoding sections from  150  to  200
Encoding sections from  200  to  250
Encoding sections from  250  to  300
Encoding sections from  300  to  350
Encoding sections from  350  to  400
Encoding sections from  400  to  450
Encoding sections from  450  to  500
Encoding sections from  500  to  506


In [ ]:
len(embeddings)

506

In [ ]:
# Convert embeddings to numpy arrays
sentence_vectors = embeddings

#Finding similarity#

In [ ]:
def calculate_similarity(input_text,sentence_vectors=sentence_vectors):
    # Encode the input text
    encoded_input = encode_sentences([input_text])

    # Get BERT embeddings for the tokenized input text
    with torch.no_grad():
        input_embedding = model(**encoded_input).last_hidden_state.mean(dim=1)  # Mean pooling of token embeddings
        input_embedding = torch.nn.functional.normalize(input_embedding, p=2, dim=-1)  # L2 normalize

    similarity_scores = cosine_similarity(input_embedding, sentence_vectors)

    # Rank sentences based on similarity scores
    ranked_sentences = sorted(zip(sectionnames, similarity_scores[0]), key=lambda x: x[1], reverse=True)

    return ranked_sentences

In [ ]:
input = "Some people trespassed my property and they hit me"
# processedinput = preprocessinput(input)

In [ ]:
calculate_similarity("A guy forced me to light a house")

[('Kidnapping ', 0.70531034),
 ('Threat of injury to induce person to refrain from applying for protection to public servant',
  0.7046249),
 ('Lurking house-trespass ', 0.701703),
 ('Punishment for cheating by personation ', 0.6981609),
 ('Deliberate and malicious acts intended to outrage religious feelings of any class by insulting its religion or religious beliefs ',
  0.69390917),
 ('Sexual harassment and punishment for sexual harassment ', 0.6919459),
 ('Punishments', 0.6888199),
 ('Criminal intimidation by an anonymous communication ', 0.68800133),
 ('Lurking house-trespass by night ', 0.6864768),
 ('Making or possession of any instrument for counterfeiting a property mark ',
  0.6861012),
 ('Punishment for bribery ', 0.68496406),
 ('Sexual intercourse by person in authority ', 0.6826535),
 ('Omission to apprehend, or sufferance of escape, on part of public servant, in cases not otherwise, provided for ',
  0.68055904),
 ('Exhibition of false light, mark or buoy ', 0.68030256),
 

In [ ]:
len(sections)

506

In [ ]:
sections[sectionnames=='False information, with intent to cause public servant to use his lawful power to the injury of another person']

array(['Whoever gives to any public servant any information which he knows or believes to be false, intending thereby to cause, or knowing it to be likely that he will thereby cause such public servant -to do or omit anything which such public servant ought not to do or omit if the true state of facts respecting which such information is given were known by him, orto use the lawful power of such public servant to the injury or annoyance of any person, shall be punished with imprisonment of either description for a term which may extend to six months, or with fine which may extend to one thousand rupees, or with both.'],
      dtype=object)

In [ ]:
embed = pd.DataFrame(sentence_vectors)
embed

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,tensor(0.0061),tensor(-0.0327),tensor(0.0146),tensor(0.0268),tensor(0.0591),tensor(-0.0094),tensor(-0.0168),tensor(-0.0109),tensor(-0.0421),tensor(0.0134),...,tensor(-0.0020),tensor(-0.0647),tensor(-0.0665),tensor(-0.0093),tensor(0.0048),tensor(0.0302),tensor(0.0568),tensor(0.0623),tensor(0.0465),tensor(0.0178)
1,tensor(0.0269),tensor(-0.0307),tensor(0.0100),tensor(0.0080),tensor(0.0596),tensor(-0.0418),tensor(-0.0333),tensor(0.0248),tensor(-0.0252),tensor(0.0060),...,tensor(-0.0047),tensor(-0.0881),tensor(-0.0404),tensor(-0.0136),tensor(0.0061),tensor(0.0099),tensor(0.0723),tensor(0.0339),tensor(0.0257),tensor(0.0262)
2,tensor(-0.0146),tensor(-0.0442),tensor(0.0169),tensor(0.0134),tensor(0.0197),tensor(-0.0304),tensor(-0.0082),tensor(0.0298),tensor(-0.0061),tensor(0.0124),...,tensor(-0.0050),tensor(-0.0912),tensor(-0.0475),tensor(-0.0235),tensor(0.0031),tensor(0.0094),tensor(0.0676),tensor(0.0185),tensor(0.0282),tensor(-0.0148)
3,tensor(-0.0094),tensor(-0.0518),tensor(0.0177),tensor(-0.0088),tensor(0.0355),tensor(-0.0175),tensor(-0.0233),tensor(0.0291),tensor(-0.0313),tensor(0.0140),...,tensor(0.0188),tensor(-0.0956),tensor(-0.0473),tensor(-0.0047),tensor(0.0165),tensor(0.0243),tensor(0.0716),tensor(0.0175),tensor(0.0415),tensor(0.0069)
4,tensor(-0.0035),tensor(-0.0278),tensor(0.0037),tensor(-0.0054),tensor(0.0640),tensor(-0.0283),tensor(-0.0263),tensor(-0.0147),tensor(-0.0306),tensor(0.0201),...,tensor(0.0006),tensor(-0.0898),tensor(-0.0296),tensor(-0.0271),tensor(0.0034),tensor(0.0196),tensor(0.0694),tensor(0.0230),tensor(0.0300),tensor(0.0173)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,tensor(-0.0077),tensor(-0.0355),tensor(0.0226),tensor(-0.0086),tensor(0.0453),tensor(-0.0197),tensor(-0.0288),tensor(0.0210),tensor(-0.0164),tensor(0.0154),...,tensor(0.0055),tensor(-0.0705),tensor(-0.0471),tensor(-0.0111),tensor(0.0169),tensor(-0.0077),tensor(0.0642),tensor(0.0165),tensor(0.0373),tensor(0.0278)
502,tensor(-0.0194),tensor(-0.0292),tensor(0.0128),tensor(-0.0146),tensor(0.0372),tensor(-0.0361),tensor(-0.0176),tensor(0.0152),tensor(-0.0178),tensor(0.0148),...,tensor(0.0087),tensor(-0.0979),tensor(-0.0382),tensor(-0.0087),tensor(0.0163),tensor(-0.0266),tensor(0.0560),tensor(0.0043),tensor(0.0538),tensor(0.0213)
503,tensor(-0.0140),tensor(0.0024),tensor(0.0328),tensor(-0.0129),tensor(0.0547),tensor(0.0062),tensor(-0.0429),tensor(0.0084),tensor(-0.0053),tensor(0.0285),...,tensor(-0.0187),tensor(-0.0547),tensor(-0.0308),tensor(-0.0109),tensor(0.0225),tensor(-0.0244),tensor(0.0490),tensor(0.0195),tensor(0.0298),tensor(0.0174)
504,tensor(-0.0203),tensor(-0.0198),tensor(0.0274),tensor(0.0020),tensor(0.0532),tensor(-0.0266),tensor(-0.0407),tensor(0.0088),tensor(-0.0047),tensor(0.0433),...,tensor(0.0026),tensor(-0.0577),tensor(-0.0396),tensor(-0.0133),tensor(0.0172),tensor(-0.0382),tensor(0.0418),tensor(0.0223),tensor(0.0262),tensor(0.0260)


In [ ]:
embed = embed.applymap(float)

In [ ]:
embed.to_csv("section_embeddings_noillustrations_24l24h",index=False)